<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/test_ci/tutorials/W2D5_GenerativeModels/W2D5_Tutorial3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuromatch Academy: Week 2, Day 5, Tutorial 3

# VAEs and GANs : Conditional GANs and Implications of GAN Technology

__Content creators:__ Kai Xu, Seungwook Han, Akash Srivastava

__Content reviewers:__ Polina Turishcheva, Melvin Selim Atay, Hadi Vafaei, Deepak Raya

__Content editors:__ Spiros Chavlis

__Production editors:__ Arush Tagade, Spiros Chavlis 

---

## Tutorial Objectives

The goal of this notebook is to understand conditional GANs. Then you will have the opportunity to experience first-hand how effective GANs are at modeling the data distribution and to question what the consequences of this technology may be.

By the end of this tutorial you will be able to:
- Understand the differences in conditional GANs.
- Generate high-dimensional natural images from a BigGAN.
- Understand the efficacy of GANs in modeling the data distribution (e.g., faces).
- Understand the energy inefficiency / environmental impact of training these large generative models.
- Understand the implications of this technology (ethics, environment, *etc*.).


In [ ]:
#@markdown Tutorial slides (pt. 1)
# you should link the slides for all tutorial videos here (we will store pdfs on osf)

from IPython.display import HTML
HTML('<iframe src="https://docs.google.com/presentation/d/1eP79mRMzD2Q7Utol3kZ5hooYIXRTQMMb" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>')

---
# Setup

In [ ]:
# Imports

# Install Huggingface BigGAN library
!pip install pytorch-pretrained-biggan --quiet
!pip install Pillow libsixel-python --quiet

# Import libraries
import nltk
import torch
import random
import torchvision

import numpy as np
import matplotlib.pyplot as plt

from pytorch_pretrained_biggan import BigGAN
from pytorch_pretrained_biggan import one_hot_from_names
from pytorch_pretrained_biggan import truncated_noise_sample

nltk.download('wordnet')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
#@title Figure settings
from IPython.display import display
import ipywidgets
import ipywidgets as widgets       # interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

In [ ]:
# @title Set seed for reproducibility in Pytorch
# https://pytorch.org/docs/stable/notes/randomness.html

def set_seed(seed):
  """
  Set random seed for reproducibility

  Args:
    seed: integer
      A positive integer to ensure reproducibility
  """
  random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  print(f'Seed {seed} has been set.')


def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


set_seed(522)

---

# Section 1: Generating with a conditional GAN (BigGAN)

In [ ]:
#@title Video 1: Conditional Generative Models
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="lV6zH2xDZck", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

In this section, we will load a pre-trained conditional GAN, BigGAN, which is the state-of-the-art model in conditional high-dimensional natural image generation, and generate samples from it. Since it is a class conditional model, we will be able to use the class label to generate images from the respective classes of objects.

Read here for more details on BigGAN: https://arxiv.org/pdf/1809.11096.pdf

In [ ]:
# Load respective BigGAN model for the specified resolution (biggan-deep-128, biggan-deep-256, biggan-deep-512)
def load_biggan(model_res):
  return BigGAN.from_pretrained('biggan-deep-{}'.format(model_res))


# Create class and noise vectors for sampling from BigGAN
def create_class_noise_vectors(class_str, trunc, num_samples):
  class_vector = one_hot_from_names([class_str]*num_samples, batch_size=num_samples)
  noise_vector = truncated_noise_sample(truncation=trunc, batch_size=num_samples)

  return class_vector, noise_vector


# Generate samples from BigGAN
def generate_biggan_samples(model, class_vector, noise_vector, truncation=0.4,
                            device=device):
  # Convert to tensor
  noise_vector = torch.from_numpy(noise_vector)
  class_vector = torch.from_numpy(class_vector)

  # Move to GPU
  noise_vector = noise_vector.to(device)
  class_vector = class_vector.to(device)
  model.to(device)

  # Generate an image
  with torch.no_grad():
      output = model(noise_vector, class_vector, truncation)

  # Back to CPU
  output = output.to('cpu')

  # The output layer of BigGAN has a tanh layer, resulting the range of [-1, 1] for the output image
  # Therefore, we normalize the images properly to [0, 1] range.
  # Clipping is only in case of numerical instability problems

  output = torch.clip(((output.detach().clone() + 1) / 2.0), 0, 1)
  output = output

  # Make grid and show generated samples
  output_grid = torchvision.utils.make_grid(output, nrow=min(4, output.shape[0]), padding=5)
  plt.imshow(output_grid.permute(1,2,0))

  return output_grid


def generate(b):
  # Create BigGAN model
  model = load_biggan(MODEL_RESOLUTION)

  # Use specified parameters (resolution, class, number of samples, etc) to generate from BigGAN
  class_vector, noise_vector = create_class_noise_vectors(CLASS, TRUNCATION, NUM_SAMPLES)
  samples_grid = generate_biggan_samples(model, class_vector, noise_vector, TRUNCATION, device)
  torchvision.utils.save_image(samples_grid, 'samples.png')
  ### If CUDA out of memory issue, lower NUM_SAMPLES (number of samples)

## Section 1.1:  Define configurations

We will now define the configurations (resolution of model, number of samples, class to sample from, truncation level) under which we will sample from BigGAN. 

***Question***: What is the truncation trick employed by BigGAN? How does sample variety and fidelity change by varying the truncation level? (Hint: play with the truncation slider and try sampling at different levels) 

In [ ]:
#@title { run: "auto" }

### RUN THIS BLOCK EVERY TIME YOU CHANGE THE PARAMETERS FOR GENERATION

# Resolution at which to generate
MODEL_RESOLUTION = "256" #@param [128, 256, 512]

# Number of images to generate
NUM_SAMPLES = 1 #@param {type:"slider", min:1, max:100, step:1}

# Class of images to generate
CLASS = 'German shepherd' #@param ['tench', 'magpie', 'jellyfish', 'German shepherd', 'bee', 'acoustic guitar', 'coffee mug', 'minibus', 'monitor']

# Truncation level of the normal distribution we sample z from
TRUNCATION = 0.4 #@param {type:"slider", min:0.1, max:1, step:0.1}

In [ ]:
# Create generate button, given parameters specified above
button = widgets.Button(description="GENERATE!",
                        layout=widgets.Layout(width='30%', height='80px'),
                        button_style='danger')
output = widgets.Output()
display(button, output)
button.on_click(generate)

# Food for thought

1. How does BigGAN differ from previous state-of-the-art generative models for high-dimensional natural images? In other words, how does BigGAN solve high-dimensional image generation? (Hint: look into model architecture and training configurations) (BigGAN paper: https://arxiv.org/pdf/1809.11096.pdf) 
2. Continuing from Question 1, what are the drawbacks of introducing such techniques into training large models for high-dimensional, diverse datasets?
3. Play with other pre-trained generative models like StyleGAN here -- where code for sampling and interpolation in the latent space is available: https://github.com/NVlabs/stylegan

---
# Section 2: Ethical issues

In [ ]:
#@markdown Tutorial slides (pt. 2)
# you should link the slides for all tutorial videos here (we will store pdfs on osf)

from IPython.display import HTML
HTML('<iframe src="https://docs.google.com/presentation/d/1-omVjYriCQumx1q_dtAGw-gWmnx7uPz5" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>')

In [ ]:
#@title Video 2: Ethical Issues
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="ZtWFeUZgfVk", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

## Section 2.1: Faces Quiz 

Now is your turn to test your abilities on recognizing a real vs. a fake image!

In [ ]:
#@markdown Real or Fake?
from IPython import display
from IPython.display import IFrame
IFrame(src='https://docs.google.com/forms/d/e/1FAIpQLSeGjn2S2bn6Q1qWjVgDS5LG7G1GsQQh2Q0T9dEUO1z5_W0yYg/viewform?usp=sf_link', width=900, height=600)

## Section 2.2: Energy Efficiency Quiz 

In [ ]:
#@markdown Make a guess
IFrame(src='https://docs.google.com/forms/d/e/1FAIpQLSe8suNt4ZmadSr_6IWq6s_nUYxC1VCpjR2cBBmQ7cR_5znCZw/viewform?usp=sf_link', width=900, height=600)

---
# Summary

In [ ]:
#@markdown Tutorial slides (pt. 3)
# you should link the slides for all tutorial videos here (we will store pdfs on osf)

from IPython.display import HTML
HTML('<iframe src="https://docs.google.com/presentation/d/1s8jHcZUudPp0h1yZLHRI9Rk8hz3zuoUS" frameborder="0" width="960" height="569" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>')

In [ ]:
#@title Video 3: Recap and advanced topics
# Insert the ID of the corresponding youtube video
from IPython.display import YouTubeVideo
video = YouTubeVideo(id="7nUjFG3N04I", width=854, height=480, fs=1)
print("Video available at https://youtu.be/" + video.id)
video

Hooray! You have finished the second week of NMA-DL course!!!

In the first section of this tutorial, we have learned: 
- How conditional GANs differ from unconditional models
- How to use a pre-trained BigGAN model to generate high-dimensional photo-realistic images and its tricks to modulate diversity and image fidelity

In the second section, we learned about the broader ethical implications of GAN technology on society through deepfakes and their tremendous energy inefficiency.

On the brighter side, as we learned throughout the week, GANs are very effective in modeling the data distribution and have many practical applications.

For example, as personalized healthcare and applications of AI in healthcare rise, the need to remove any Personally Identifiable Information (PII) becomes more important. As shown in this paper (https://link.springer.com/chapter/10.1007/978-3-030-45385-5_36), GANs can be leveraged to anonymize healthcare data.

As a food for thought, what are some other practical applications of GANs that you can think of? Discuss with your pod your ideas.